In [48]:
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
from html2image import Html2Image
from tqdm import tqdm
import os
import shutil

In [82]:
comments_file_path = "comments.html"
youtubestrap_file_path = "youtubestrap.html"
index_file_path = "index.html"

content_tag_id = "HEREISCONTENT"

hti = Html2Image()

In [83]:
# Read comments from comments.html file created by user
with open(comments_file_path, "r", encoding="utf8") as f:
	comments_file_content = f.read()
 
comments_html = BeautifulSoup(comments_file_content)
comments_html_list = comments_html("ytd-comment-renderer")

In [85]:
# Read the contents of the youtubestrap.html file and setup styles and new content tag
with open(youtubestrap_file_path, "r", encoding="utf8") as f:
    youtubestrap = f.read()
    
youtubestrap_html = BeautifulSoup(youtubestrap)
youtubestrap_head_html = youtubestrap_html.find("head")

youtubestrap_styles_html = youtubestrap_head_html.children
youtubestrap_body_html = youtubestrap_html.find("body")

In [87]:
# download a yt webpage to change
url = "https://www.youtube.com/watch?v=1-emQo-7O3Y"

response = urllib.request.urlopen(url)
index_file_content = response.read()

In [88]:
os.makedirs("out", exist_ok=True)

loop = tqdm(comments_html_list)

index_html = BeautifulSoup(index_file_content)
head_tag = index_html.find("head")
head_tag.extend(list(youtubestrap_styles_html))

content_tag = youtubestrap_body_html.find("div", {"id": content_tag_id})

for i, comment in enumerate(loop):
	content_tag.clear()
	content_tag.append(comment)

	body_tag = index_html.find("body")
	body_tag.replaceWith(youtubestrap_body_html)

	with open(index_file_path, "w", encoding="utf8") as f:
		f.write(index_html.decode(pretty_print=False, eventual_encoding="utf8"))
  
	fname = f'{i}.png'
	out_path = os.path.join("out", fname)
	hti.screenshot(html_file=index_file_path , save_as=fname, size=(500, 200))
	shutil.move(fname, out_path) 

  0%|          | 0/359 [00:14<?, ?it/s]
